# KITTI Odometry Dataset

## Import

In [33]:
import os
from io import TextIOWrapper
import sys
import struct
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
from scipy.spatial.transform import Rotation
from pathlib import Path
from glob import glob
KITTI_ROOT = Path('/data/datasets/dataset_kitti')
KITTI_SEQS = ['00', '01', '02',
              '03', '04', '05', 
              '06', '07', '08',
              '09', '10']

## Rearrage LiDAR

In [48]:
def rearrage_calib(root:Path, seq:str):
    old_calib_fn = root / 'data_odometry_calib' / 'dataset' / 'sequences' / seq / 'calib.txt'
    old_times_fn = root / 'data_odometry_calib' / 'dataset' / 'sequences' / seq / 'times.txt'
    new_calib_fn = root / seq / 'raw_calib.txt'
    new_times_fn = root / seq / 'raw_times.txt'
    if new_calib_fn.exists() :
        return False
    if new_times_fn.exists():
        return False
    
    # Rename
    os.rename(old_calib_fn, new_calib_fn)
    os.rename(old_times_fn, new_times_fn)
    return True

def rearrage_poses(root:Path, seq:str):
    old_poses_fn = root / 'data_odometry_poses' / 'dataset' / 'poses' / f'{seq}.txt'
    new_poses_fn = root / seq / 'raw_poses.txt'
    if new_poses_fn.exists():
        return False
    
    os.rename(old_poses_fn, new_poses_fn)
    return True

def rearrage_lidar(root:Path, seq:str):
    old_lidar_dir = root / seq / 'velodyne'
    new_lidar_dir = root / seq / 'lidar'
    old_paths = glob(f'{old_lidar_dir}/*.bin')
    new_paths = glob(f'{new_lidar_dir}/*.bin')
    old_N = len(old_paths)
    new_N = len(new_paths)
    if new_N != 0:
        print('Lidar is already re-arranged')
        return False
    
    old_paths.sort()
    os.makedirs(new_lidar_dir, exist_ok=True)
    for old_path in old_paths:
        old_fn = Path(old_path)
        new_fn = new_lidar_dir / old_fn.name
        print(new_fn)
        os.rename(old_fn, new_fn)
        
    return True

def rearrage_data(root:Path, seq:str):
    if not root.exists():
        print(f"{root} doesn't exist!")
        return False
    
    if not (root / seq ).exists():
        os.makedirs(root/seq)

    rearrage_calib(root, seq)
    rearrage_poses(root, seq)
    rearrage_lidar(root, seq)
    calib_fn = root / seq / 'raw_calib.txt'
    times_fn = root / seq / 'raw_times.txt'
    poses_fn = root / seq / 'raw_poses.txt'
    # calib = np.genfromtxt(calib_fn, dtype=float)
    times = np.genfromtxt(times_fn, dtype=float)
    poses = np.genfromtxt(poses_fn, dtype=float)
    print('timestamps: ', times.shape)
    print('poses: ', poses.shape)
    
    lidar_dir = root / seq / 'lidar'
    lidar_fns = glob(f'{lidar_dir}/*.bin')
    lidar_fns.sort()
    N_SCANS = len(lidar_fns)
    print(f"N_SCANS: {N_SCANS}")
    
    assert (N_SCANS == times.shape[0])
    assert (N_SCANS == poses.shape[0])
    
    data_pose_fn = root / seq / 'pose.txt'
    f_pose = open(data_pose_fn, 'w')
    for i in range(N_SCANS):
        time = times[i]
        pose = poses[i]
        rot_mat = np.array([pose[0:3],
                            pose[4:7],
                            pose[8:11]])
        # print(rot_mat.shape)
        rot_mat = Rotation.from_matrix(rot_mat)
        quat = rot_mat.as_quat() # x, y, z, w
        # print(quat.shape)
        trans = pose[3::4]
        # print(trans.shape)
        line = f"{time:.6f} {trans[0]:.6f} {trans[1]:.6f} {trans[2]:.6f} {quat[0]:.6f} {quat[1]:.6f} {quat[2]:.6f} {quat[3]:.6f}\n"
        f_pose.write(line)
    f_pose.close()
    
    lidar_data_fn = root / seq / 'lidar_data.txt'
    f_lidar = open(lidar_data_fn, 'w')
    for i in range(N_SCANS):
        time = times[i]
        lidar_fn = Path(lidar_fns[i])
        line = f"{time:.6f} lidar/{lidar_fn.name}\n"
        f_lidar.write(line)
    f_lidar.close()
    
    return True
        
        
    
    
    old_pose_fn = root / seq / 'old_pose.txt'  
    
    # Rename
    os.rename(pose_fn, old_pose_fn)
    
    
    # Parse Timestamps
    timestamps = np.genfromtxt(old_times_fn)
    print('timestamps: ', timestamps.shape)
    
    
    f_pose = open(pose_fn, 'r')
    f_data = open(lidar_data_fn, 'w')
        
    os.makedirs(new_lidar_dir, exist_ok=True)
    poses = f_pose.readlines()
    
    
    scan_paths = glob(f"{old_lidar_dir}/*.bin")
    scan_paths.sort()
    assert(len(scan_paths) == len(poses))
    
    for i in range(len(scan_paths)):
        print(scan_paths[i])
        print(poses[i])
        
        words = poses[i].split(' ')
        t = float(words[0])
        px = float(words[1])
        py = float(words[2])
        pz = float(words[3])
        
        if i == 5:
            break
    
    return True
    
    # timestamps = []
    # for scan_path in scan_paths:
    #     timestamp = float(Path(scan_path).name.split('.')[0]) / 1e6
    #     timestamps.append(timestamp)
    #     if len(timestamps) >= 2:
    #         if timestamps[-2] >= timestamps[-1]:
    #             print("Time order is wrong!")
    #             return False
    
    # assert (len(timestamps) == len(scan_paths))
    
    # for index in range(len(timestamps)):
    #     line = f"{timestamps[index]:.6f} lidar/{index:06d}.bin\n"
    #     old_scan_fn = scan_paths[index]
    #     new_scan_fn = f"{new_lidar_dir}/{index:06d}.bin"
    #     # print(old_scan_fn)
    #     # print(new_scan_fn)
    #     os.rename(old_scan_fn, new_scan_fn)
    #     f_data.write(line)

    # f_data.close()
    # return True

In [50]:
for seq in KITTI_SEQS:
    rearrage_data(KITTI_ROOT, seq)

Lidar is already re-arranged
timestamps:  (4541,)
poses:  (4541, 12)
N_SCANS: 4541
Lidar is already re-arranged
timestamps:  (1101,)
poses:  (1101, 12)
N_SCANS: 1101
/data/datasets/dataset_kitti/02/lidar/000000.bin
/data/datasets/dataset_kitti/02/lidar/000001.bin
/data/datasets/dataset_kitti/02/lidar/000002.bin
/data/datasets/dataset_kitti/02/lidar/000003.bin
/data/datasets/dataset_kitti/02/lidar/000004.bin
/data/datasets/dataset_kitti/02/lidar/000005.bin
/data/datasets/dataset_kitti/02/lidar/000006.bin
/data/datasets/dataset_kitti/02/lidar/000007.bin
/data/datasets/dataset_kitti/02/lidar/000008.bin
/data/datasets/dataset_kitti/02/lidar/000009.bin
/data/datasets/dataset_kitti/02/lidar/000010.bin
/data/datasets/dataset_kitti/02/lidar/000011.bin
/data/datasets/dataset_kitti/02/lidar/000012.bin
/data/datasets/dataset_kitti/02/lidar/000013.bin
/data/datasets/dataset_kitti/02/lidar/000014.bin
/data/datasets/dataset_kitti/02/lidar/000015.bin
/data/datasets/dataset_kitti/02/lidar/000016.bin
/